In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import textwrap

from langchain.llms import OpenAI
MODEL = "text-davinci-003"
llm = OpenAI(temperature=0, model_name=MODEL)

In [ ]:
# PromptTemplate
from langchain import PromptTemplate
simple_answer_prompt = PromptTemplate.from_template("""
Answer the question below. Some information is provided. 

Question: {question}

Relevant Information: {relevant_information}
""")

final_prompt = simple_answer_prompt.format(question="What is the capital of France?", relevant_information="France is a country in Europe.")

print(final_prompt)
print("--------------------------------------------------")
print(llm(final_prompt))

In [ ]:
# chat
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

MODEL = "gpt-3.5-turbo"

chat = ChatOpenAI(temperature=0, model_name=MODEL)
history = ChatMessageHistory()

QUESTION = "Who holds the record in 400-meter individual medley?"
print(f"Question: {textwrap.fill(QUESTION, 50)}")
history.add_user_message(QUESTION)

response = chat(history.messages)

print(f"Answer: {textwrap.fill(response.content, 50)}")
history.add_ai_message(response.content)

print('\n\n')
QUESTION = "But I recently read Leon Marchand got 4:02.50 in 400-meter individual medley"
print(f"Question: {textwrap.fill(QUESTION, 50)}")
history.add_user_message(QUESTION)

response = chat(history.messages)
print(f"Answer: {textwrap.fill(response.content, 50)}")


In [ ]:
# chains
from download_pdf import download_pdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
file_name = os.getcwd() + "/pdfs/" + os.getenv("SUMMARIZE_FILE_NAME")
download_pdf(os.getenv("SUMMARIZE_FILE_URL"), file_name)



loader = PyPDFLoader(file_name)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
print(textwrap.fill(chain.run(texts), 50))



In [ ]:
# agents
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import Tool

def substract_tool(input):
    x, y = input.split("-")
    return str(int(x) - int(y))

toolkit = load_tools(["ddg-search"], llm=llm)
toolkit.append(Tool(
    name="substraction-tool",
    description="Substracts two numbers of the form 'x-y' and returns the result.",
    func=substract_tool
))

agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
response = agent({"input":"what was the difference between the average temperature in vancouver canada summer vs winter"})